# 서울시 교통사고 시각화

In [3]:
import pandas as pd

In [11]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [12]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]

In [13]:
# index 정리하기
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [14]:
traffic_Seoul_2019.set_index('자치구명', inplace=True)
traffic_Seoul_2019

,년도,월,발생건수,사망자수,부상자수
자치구명,,,,,
종로구,2019,1,87,1,125
종로구,2019,2,66,1,84
종로구,2019,3,87,2,122
종로구,2019,4,85,0,131
종로구,2019,5,112,1,158
...,...,...,...,...,...
강동구,2019,8,127,2,175
강동구,2019,9,98,2,137
강동구,2019,10,108,1,144


In [18]:
import numpy as np

In [20]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
#traffic_analysis = traffic_Seoul_2019[['발생건수','부상자수','사망자수']].groupby('자치구명').sum()
traffic_analysis = pd.pivot_table(traffic_Seoul_2019[['발생건수','부상자수','사망자수']], index='자치구명', aggfunc=np.sum)
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [26]:
# 위도, 경도 데이터 불러오기
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding="euc-Kr")
seoul_limit.columns = ['자치구명','lon','lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [33]:
# traffic_Seoul_2019와 seoul_limit 병합하기

data_result = pd.merge(traffic_analysis, seoul_limit, on='자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [34]:
data_result.tail()

,자치구명,발생건수,부상자수,사망자수,lon,lat
20,용산구,1187,1614,10,126.9904,37.53254
21,은평구,1119,1464,7,126.9292,37.60278
22,종로구,1133,1559,6,126.9793,37.57323
23,중구,1184,1615,5,127.0032,37.55273
24,중랑구,1620,2122,12,127.0928,37.60652


In [35]:
# folium은 int를 읽지못함 float만 가능
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [38]:
# folium을 사용하기 위해 int를 float로 변환해야 한다.

data_result[['발생건수','부상자수','사망자수']] = data_result[['발생건수','부상자수','사망자수']].astype(float)

In [116]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.9+ KB


### 자치구의 구청을 표시하기

In [164]:
import folium

In [165]:
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11,
    
)
# 자치구별 교통사고 발생건수를 표시하기(Marker)

# for n in data_result.index:
#    folium.Marker([data_result['lat'][n],data_result['lon'][n]],
#                   popup=data_result['자치구명'][n]
#                   ).add_to(map)
#    folium.CircleMarker(
#    [data_result['lat'][n],data_result['lon'][n]],
#    radius=data_result['발생건수'][n] / 100,  # 원의 반지름
#    color = 'cornflowerblue',
#    fill = False,
#    fill_color='cornflowerblue',
#    fill_opacity=0.3,
#    ).add_to(map)

# map


# 자치구별 교통사고 사망건수를 표시하기

In [133]:
for n in data_result.index:
    folium.Marker([data_result['lat'][n],data_result['lon'][n]],
                popup=data_result['자치구명'][n]
                ).add_to(map)
    folium.CircleMarker(
        [data_result['lat'][n],data_result['lon'][n]],
        radius=data_result['사망자수'][n] * 1.9,
        color = 'red',
        fill = False,
        fill_color='red',
        fill_opacity=0.3,
    ).add_to(map)
map

In [153]:
data_result.sort_values('사망자수', inplace=True)
data_result.reset_index(drop=True, inplace=True)
data_result

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,광진구,973.0,1316.0,3.0,127.0824,37.53862
1,도봉구,852.0,1163.0,4.0,127.0478,37.66873
2,동작구,1406.0,2021.0,5.0,126.9393,37.51243
3,중구,1184.0,1615.0,5.0,127.0032,37.55273
4,종로구,1133.0,1559.0,6.0,126.9793,37.57323
5,마포구,1494.0,2123.0,7.0,126.9016,37.56631
6,강북구,1277.0,1706.0,7.0,127.0255,37.63975
7,은평구,1119.0,1464.0,7.0,126.9292,37.60278
8,노원구,1703.0,2302.0,8.0,127.0565,37.65437
9,성북구,1509.0,2007.0,8.0,127.0167,37.58936


In [167]:
folium.Marker([data_result['lat'][0],data_result['lon'][0]],
            popup=data_result['자치구명'][0]
            ).add_to(map)
folium.CircleMarker(
    [data_result['lat'][0],data_result['lon'][0]],
    radius=data_result['사망자수'][0] * 1.9,
    color = 'red',
    fill = False,
    fill_color='red',
    fill_opacity=0.3,
).add_to(map)

folium.Marker([data_result['lat'][24],data_result['lon'][24]],
            popup=data_result['자치구명'][24]
            ).add_to(map)
folium.CircleMarker(
    [data_result['lat'][24],data_result['lon'][24]],
    radius=data_result['사망자수'][24] * 1.9,
    color = 'red',
    fill = False,
    fill_color='red',
    fill_opacity=0.3,
).add_to(map)

map

0     37.53862
24    37.55094
Name: lat, dtype: float64